In [65]:
#importacion de la biblioteca requests

import requests

In [66]:
#scrappeamos la direccion url requerida

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [67]:
data = requests.get(standings_url)

In [68]:
#utilizamos beautifulSoup para nuestro estudio

from bs4 import BeautifulSoup

In [69]:
#necesario para poder scrappear la tabla

soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [70]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [71]:
data = requests.get(team_urls[0])

In [72]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [73]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [74]:
data = requests.get(f"https://fbref.com{links[0]}")

In [75]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [76]:
shooting.head()

For Arsenal                                                              \
         Date   Time            Comp        Round  Day Venue Result GF GA   
0  2022-08-05  20:00  Premier League  Matchweek 1  Fri  Away      W  2  0   
1  2022-08-13  15:00  Premier League  Matchweek 2  Sat  Home      W  4  2   
2  2022-08-20  17:30  Premier League  Matchweek 3  Sat  Away      W  3  0   
3  2022-08-27  17:30  Premier League  Matchweek 4  Sat  Home      W  2  1   
4  2022-08-31  19:30  Premier League  Matchweek 5  Wed  Home      W  2  1   

                   ... Standard               Expected                    \
         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  Crystal Palace  ...     14.6  1.0  0     0      1.0  1.0    0.10  0.0   
1  Leicester City  ...     13.0  0.0  0     0      2.7  2.7    0.16  1.3   
2     Bournemouth  ...     14.8  0.0  0     0      1.3  1.3    0.10  1.7   
3          Fulham  ...     15.5  1.0  0     0      2.6  2.6    0.12 -0.6   
4     Aston Villa  ...     16.3  1.0  0     0      2.4  2.4    0.12 -0.4   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0     0.0        Match Report  
1     1.3        Match Report  
2     1.7        Match Report  
3    -0.6        Match Report  
4    -0.4        Match Report  

[5 rows x 26 columns]

In [77]:
#con esto eliminamos la cabecera de For Arsenal

shooting.columns = shooting.columns.droplevel()

In [78]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [79]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,4-3-3,Anthony Taylor,Match Report,NaN,10,2,14.6,1.0,0,0
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,4-3-3,Darren England,Match Report,NaN,19,7,13.0,0.0,0,0
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,4-3-3,Craig Pawson,Match Report,NaN,14,6,14.8,0.0,0,0
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,4-3-3,Jarred Gillett,Match Report,NaN,22,8,15.5,1.0,0,0
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,4-3-3,Robert Jones,Match Report,NaN,22,8,16.3,1.0,0,0


In [80]:
years = list(range(2022, 2020, -1))
all_matches = []

In [81]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [82]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [83]:
len(all_matches)

40

In [84]:
match_df = pd.concat(all_matches)

In [85]:
match_df.columns = [c.lower() for c in match_df.columns]

In [104]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Away,L,0,2,Aston Villa,...,Match Report,NaN,9.0,3.0,21.6,0.0,0.0,0.0,2021,Norwich City
39,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,L,0,4,West Ham,...,Match Report,NaN,8.0,2.0,22.2,1.0,0.0,0.0,2021,Norwich City
40,2022-05-11,19:45,Premier League,Matchweek 21,Wed,Away,L,0,3,Leicester City,...,Match Report,NaN,9.0,5.0,17.0,0.0,0.0,0.0,2021,Norwich City
41,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Away,D,1,1,Wolves,...,Match Report,NaN,11.0,2.0,14.4,0.0,0.0,0.0,2021,Norwich City


In [105]:
match_df.to_csv("matches.csv")

In [172]:
matches = pd.read_csv("matches.csv", index_col=0)

In [173]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal


In [174]:
matches.shape

(1200, 27)

In [175]:
# 2 seasons * 20 squads * 38 matches

1 * 20 * 38

760

In [176]:
# Missing Arsenal 2022-2023
matches["team"].value_counts()

Manchester United           61
Tottenham Hotspur           61
Crystal Palace              60
Wolverhampton Wanderers     60
Newcastle United            60
Southampton                 60
Everton                     60
Brentford                   60
Leeds United                60
Chelsea                     60
Aston Villa                 60
Manchester City             60
Leicester City              60
West Ham United             60
Arsenal                     59
Liverpool                   59
Brighton and Hove Albion    59
Burnley                     38
Watford                     38
Norwich City                38
Fulham                      23
Nottingham Forest           22
Bournemouth                 22
Name: team, dtype: int64

In [177]:
matches[matches["team"] == "Arsenal"].sort_values("date").reset_index(drop=True)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2021-08-13,20:00,Premier League,Matchweek 1,Fri,Away,L,0.0,2.0,Brentford,...,Match Report,NaN,22.0,4.0,19.0,1.0,0.0,0.0,2021,Arsenal
1,2021-08-22,16:30,Premier League,Matchweek 2,Sun,Home,L,0.0,2.0,Chelsea,...,Match Report,NaN,6.0,3.0,18.7,0.0,0.0,0.0,2021,Arsenal
2,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Away,L,0.0,5.0,Manchester City,...,Match Report,NaN,1.0,0.0,6.8,0.0,0.0,0.0,2021,Arsenal
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,...,Match Report,NaN,30.0,6.0,16.9,1.0,0.0,0.0,2021,Arsenal
4,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,...,Match Report,NaN,13.0,3.0,20.7,1.0,0.0,0.0,2021,Arsenal
5,2021-09-26,16:30,Premier League,Matchweek 6,Sun,Home,W,3.0,1.0,Tottenham,...,Match Report,NaN,12.0,7.0,17.7,0.0,0.0,0.0,2021,Arsenal
6,2021-10-02,17:30,Premier League,Matchweek 7,Sat,Away,D,0.0,0.0,Brighton,...,Match Report,NaN,8.0,2.0,21.1,1.0,0.0,0.0,2021,Arsenal
7,2021-10-18,20:00,Premier League,Matchweek 8,Mon,Home,D,2.0,2.0,Crystal Palace,...,Match Report,NaN,17.0,6.0,17.6,1.0,0.0,0.0,2021,Arsenal
8,2021-10-22,20:00,Premier League,Matchweek 9,Fri,Home,W,3.0,1.0,Aston Villa,...,Match Report,NaN,21.0,8.0,17.3,1.0,0.0,1.0,2021,Arsenal
9,2021-10-30,12:30,Premier League,Matchweek 10,Sat,Away,W,2.0,0.0,Leicester City,...,Match Report,NaN,9.0,5.0,16.7,0.0,0.0,0.0,2021,Arsenal


In [178]:
matches["round"].value_counts()

Matchweek 1     40
Matchweek 14    40
Matchweek 23    40
Matchweek 2     40
Matchweek 21    40
Matchweek 20    40
Matchweek 19    40
Matchweek 18    40
Matchweek 17    40
Matchweek 16    40
Matchweek 15    40
Matchweek 22    40
Matchweek 13    40
Matchweek 10    40
Matchweek 9     40
Matchweek 6     40
Matchweek 5     40
Matchweek 4     40
Matchweek 3     40
Matchweek 11    40
Matchweek 12    38
Matchweek 8     36
Matchweek 7     26
Matchweek 37    20
Matchweek 33    20
Matchweek 32    20
Matchweek 36    20
Matchweek 35    20
Matchweek 34    20
Matchweek 30    20
Matchweek 26    20
Matchweek 31    20
Matchweek 29    20
Matchweek 28    20
Matchweek 27    20
Matchweek 25    20
Matchweek 24    20
Matchweek 38    20
Name: round, dtype: int64

In [179]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [180]:
del matches["comp"]

In [181]:
del matches["notes"]

In [182]:
matches["date"] = pd.to_datetime(matches["date"])

In [183]:
matches["target"] = (matches["result"] == "W").astype("int")

In [184]:
matches

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,match report,sh,sot,dist,fk,pk,pkatt,season,team,target
0,2022-08-05,20:00,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,1.0,...,Match Report,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal,1
1,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,2.7,...,Match Report,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal,1
2,2022-08-20,17:30,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,1.3,...,Match Report,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal,1
3,2022-08-27,17:30,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,2.6,...,Match Report,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal,1
4,2022-08-31,19:30,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,2.4,...,Match Report,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2022-04-30,15:00,Matchweek 35,Sat,Away,L,0.0,2.0,Aston Villa,0.5,...,Match Report,9.0,3.0,21.6,0.0,0.0,0.0,2021,Norwich City,0
39,2022-05-08,14:00,Matchweek 36,Sun,Home,L,0.0,4.0,West Ham,0.8,...,Match Report,8.0,2.0,22.2,1.0,0.0,0.0,2021,Norwich City,0
40,2022-05-11,19:45,Matchweek 21,Wed,Away,L,0.0,3.0,Leicester City,1.1,...,Match Report,9.0,5.0,17.0,0.0,0.0,0.0,2021,Norwich City,0
41,2022-05-15,14:00,Matchweek 37,Sun,Away,D,1.0,1.0,Wolves,1.1,...,Match Report,11.0,2.0,14.4,0.0,0.0,0.0,2021,Norwich City,0


In [185]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [186]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [187]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [188]:
matches["day_code"] = matches["date"].dt.dayofweek

In [189]:
matches

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,fk,pk,pkatt,season,team,target,venue_code,opp_code,hour,day_code
0,2022-08-05,20:00,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,1.0,...,1.0,0.0,0.0,2022,Arsenal,1,0,7,20,4
1,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,2.7,...,0.0,0.0,0.0,2022,Arsenal,1,1,11,15,5
2,2022-08-20,17:30,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,1.3,...,0.0,0.0,0.0,2022,Arsenal,1,0,2,17,5
3,2022-08-27,17:30,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,2.6,...,1.0,0.0,0.0,2022,Arsenal,1,1,9,17,5
4,2022-08-31,19:30,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,2.4,...,1.0,0.0,0.0,2022,Arsenal,1,1,1,19,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2022-04-30,15:00,Matchweek 35,Sat,Away,L,0.0,2.0,Aston Villa,0.5,...,0.0,0.0,0.0,2021,Norwich City,0,0,1,15,5
39,2022-05-08,14:00,Matchweek 36,Sun,Home,L,0.0,4.0,West Ham,0.8,...,1.0,0.0,0.0,2021,Norwich City,0,1,21,14,6
40,2022-05-11,19:45,Matchweek 21,Wed,Away,L,0.0,3.0,Leicester City,1.1,...,0.0,0.0,0.0,2021,Norwich City,0,0,11,19,2
41,2022-05-15,14:00,Matchweek 37,Sun,Away,D,1.0,1.0,Wolves,1.1,...,0.0,0.0,0.0,2021,Norwich City,0,0,22,14,6


In [190]:
from sklearn.ensemble import RandomForestClassifier

In [191]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [192]:
train = matches[matches["date"] < '2023-01-01']

In [193]:
test = matches[matches["date"] > '2023-01-01']

In [194]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [195]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [196]:
preds = rf.predict(test[predictors])

In [197]:
from sklearn.metrics import accuracy_score

In [198]:
error = accuracy_score(test["target"], preds)

In [199]:
error

0.5833333333333334

In [200]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [201]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,52,18
1,27,11


In [202]:
from sklearn.metrics import precision_score

precision_score(test["target"], preds)

0.3793103448275862

In [203]:
grouped_matches = matches.groupby("team")

In [204]:
group = grouped_matches.get_group("Arsenal").sort_values("date")

In [205]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [206]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
4,2021-09-11,15:00,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,2.7,...,15,5,0.000000,3.000000,9.666667,2.333333,14.833333,0.333333,0.000000,0.000000
5,2021-09-18,15:00,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,1.1,...,15,5,0.333333,2.333333,12.333333,3.000000,14.133333,0.333333,0.000000,0.000000
7,2021-09-26,16:30,Matchweek 6,Sun,Home,W,3.0,1.0,Tottenham,1.1,...,16,6,0.666667,1.666667,14.666667,3.000000,14.800000,0.666667,0.000000,0.000000
8,2021-10-02,17:30,Matchweek 7,Sat,Away,D,0.0,0.0,Brighton,0.4,...,17,5,1.666667,0.333333,18.333333,5.333333,18.433333,0.666667,0.000000,0.000000
9,2021-10-18,20:00,Matchweek 8,Mon,Home,D,2.0,2.0,Crystal Palace,1.7,...,20,0,1.333333,0.333333,11.000000,4.000000,19.833333,0.666667,0.000000,0.000000
10,2021-10-22,20:00,Matchweek 9,Fri,Home,W,3.0,1.0,Aston Villa,2.7,...,20,4,1.666667,1.000000,12.333333,5.000000,18.800000,0.666667,0.000000,0.000000
12,2021-10-30,12:30,Matchweek 10,Sat,Away,W,2.0,0.0,Leicester City,0.7,...,12,5,1.666667,1.000000,15.333333,5.333333,18.666667,1.000000,0.000000,0.333333
13,2021-11-07,14:00,Matchweek 11,Sun,Home,W,1.0,0.0,Watford,1.6,...,14,6,2.333333,1.000000,15.666667,6.333333,17.200000,0.666667,0.000000,0.333333
14,2021-11-20,17:30,Matchweek 12,Sat,Away,L,0.0,4.0,Liverpool,0.4,...,17,5,2.000000,0.333333,14.333333,6.000000,18.366667,0.666667,0.000000,0.666667
15,2021-11-27,12:30,Matchweek 13,Sat,Home,W,2.0,0.0,Newcastle Utd,2.5,...,12,5,1.000000,1.333333,9.000000,4.333333,19.233333,0.333333,0.000000,0.333333


In [207]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [208]:
matches_rolling

date   time         round  day venue result  \
team                                                                           
Arsenal                 4  2021-09-11  15:00   Matchweek 4  Sat  Home      W   
                        5  2021-09-18  15:00   Matchweek 5  Sat  Away      W   
                        7  2021-09-26  16:30   Matchweek 6  Sun  Home      W   
                        8  2021-10-02  17:30   Matchweek 7  Sat  Away      D   
                        9  2021-10-18  20:00   Matchweek 8  Mon  Home      D   
...                               ...    ...           ...  ...   ...    ...   
Wolverhampton Wanderers 20 2023-01-04  20:00  Matchweek 19  Wed  Away      D   
                        23 2023-01-14  15:00  Matchweek 20  Sat  Home      W   
                        25 2023-01-22  14:00  Matchweek 21  Sun  Away      L   
                        26 2023-02-04  15:00  Matchweek 22  Sat  Home      W   
                        27 2023-02-11  15:00  Matchweek 23  Sat  Away      W   

                             gf   ga         opponent   xg  ...  hour  \
team                                                        ...         
Arsenal                 4   1.0  0.0     Norwich City  2.7  ...    15   
                        5   1.0  0.0          Burnley  1.1  ...    15   
                        7   3.0  1.0        Tottenham  1.1  ...    16   
                        8   0.0  0.0         Brighton  0.4  ...    17   
                        9   2.0  2.0   Crystal Palace  1.7  ...    20   
...                         ...  ...              ...  ...  ...   ...   
Wolverhampton Wanderers 20  1.0  1.0      Aston Villa  0.9  ...    20   
                        23  1.0  0.0         West Ham  0.9  ...    15   
                        25  0.0  3.0  Manchester City  0.8  ...    14   
                        26  3.0  0.0        Liverpool  2.0  ...    15   
                        27  2.0  1.0      Southampton  0.8  ...    15   

                            day_code  gf_rolling ga_rolling sh_rolling  \
team                                                                     
Arsenal                 4          5    0.000000   3.000000   9.666667   
                        5          5    0.333333   2.333333  12.333333   
                        7          6    0.666667   1.666667  14.666667   
                        8          5    1.666667   0.333333  18.333333   
                        9          0    1.333333   0.333333  11.000000   
...                              ...         ...        ...        ...   
Wolverhampton Wanderers 20         2    0.666667   1.333333   9.333333   
                        23         5    1.000000   1.000000   9.000000   
                        25         6    0.666667   0.666667  12.333333   
                        26         5    0.666667   1.333333  12.333333   
                        27         5    1.333333   1.000000  13.000000   

                           sot_rolling dist_rolling  fk_rolling  pk_rolling  \
team                                                                          
Arsenal                 4     2.333333    14.833333    0.333333         0.0   
                        5     3.000000    14.133333    0.333333         0.0   
                        7     3.000000    14.800000    0.666667         0.0   
                        8     5.333333    18.433333    0.666667         0.0   
                        9     4.000000    19.833333    0.666667         0.0   
...                                ...          ...         ...         ...   
Wolverhampton Wanderers 20    3.000000    18.633333    1.000000         0.0   
                        23    3.333333    16.700000    0.666667         0.0   
                        25    3.333333    17.166667    0.666667         0.0   
                        26    2.666667    16.866667    0.000000         0.0   
                        27    3.333333    18.233333    0.000000         0.0   

                            pkatt_rolling  
team   

In [209]:
matches_rolling = matches_rolling.droplevel('team')

In [210]:
matches_rolling

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
4,2021-09-11,15:00,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,2.7,...,15,5,0.000000,3.000000,9.666667,2.333333,14.833333,0.333333,0.0,0.0
5,2021-09-18,15:00,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,1.1,...,15,5,0.333333,2.333333,12.333333,3.000000,14.133333,0.333333,0.0,0.0
7,2021-09-26,16:30,Matchweek 6,Sun,Home,W,3.0,1.0,Tottenham,1.1,...,16,6,0.666667,1.666667,14.666667,3.000000,14.800000,0.666667,0.0,0.0
8,2021-10-02,17:30,Matchweek 7,Sat,Away,D,0.0,0.0,Brighton,0.4,...,17,5,1.666667,0.333333,18.333333,5.333333,18.433333,0.666667,0.0,0.0
9,2021-10-18,20:00,Matchweek 8,Mon,Home,D,2.0,2.0,Crystal Palace,1.7,...,20,0,1.333333,0.333333,11.000000,4.000000,19.833333,0.666667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2023-01-04,20:00,Matchweek 19,Wed,Away,D,1.0,1.0,Aston Villa,0.9,...,20,2,0.666667,1.333333,9.333333,3.000000,18.633333,1.000000,0.0,0.0
23,2023-01-14,15:00,Matchweek 20,Sat,Home,W,1.0,0.0,West Ham,0.9,...,15,5,1.000000,1.000000,9.000000,3.333333,16.700000,0.666667,0.0,0.0
25,2023-01-22,14:00,Matchweek 21,Sun,Away,L,0.0,3.0,Manchester City,0.8,...,14,6,0.666667,0.666667,12.333333,3.333333,17.166667,0.666667,0.0,0.0
26,2023-02-04,15:00,Matchweek 22,Sat,Home,W,3.0,0.0,Liverpool,2.0,...,15,5,0.666667,1.333333,12.333333,2.666667,16.866667,0.000000,0.0,0.0


In [211]:
matches_rolling.index = range(matches_rolling.shape[0])

In [212]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2023-01-01']
    test = data[data["date"] > '2023-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [213]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [214]:
error

0.46153846153846156

In [215]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [216]:
combined.head(10)

,actual,predicted,date,team,opponent,result
51,0,1,2023-01-03,Arsenal,Newcastle Utd,D
52,1,1,2023-01-15,Arsenal,Tottenham,W
53,1,1,2023-01-22,Arsenal,Manchester Utd,W
54,0,1,2023-02-04,Arsenal,Everton,L
55,0,1,2023-02-11,Arsenal,Brentford,D
108,0,0,2023-01-04,Aston Villa,Wolves,D
109,1,0,2023-01-13,Aston Villa,Leeds United,W
110,1,0,2023-01-21,Aston Villa,Southampton,W
111,0,1,2023-02-04,Aston Villa,Leicester City,L
112,0,0,2023-02-12,Aston Villa,Manchester City,L


In [217]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)


In [218]:
combined["new_team"] = combined["team"].map(mapping)

In [219]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [220]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,0,1,2023-01-03,Arsenal,Newcastle Utd,D,Arsenal,0,0,Newcastle United,Arsenal,D,Newcastle Utd
1,1,1,2023-01-15,Arsenal,Tottenham,W,Arsenal,0,0,Tottenham Hotspur,Arsenal,L,Tottenham
2,1,1,2023-01-22,Arsenal,Manchester Utd,W,Arsenal,0,1,Manchester United,Arsenal,L,Manchester Utd
3,0,1,2023-02-04,Arsenal,Everton,L,Arsenal,1,0,Everton,Arsenal,W,Everton
4,0,1,2023-02-11,Arsenal,Brentford,D,Arsenal,0,0,Brentford,Arsenal,D,Brentford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0,0,2023-01-04,Wolverhampton Wanderers,Aston Villa,D,Wolves,0,0,Aston Villa,Wolves,D,Aston Villa
99,1,0,2023-01-14,Wolverhampton Wanderers,West Ham,W,Wolves,0,0,West Ham United,Wolves,L,West Ham
100,0,0,2023-01-22,Wolverhampton Wanderers,Manchester City,L,Wolves,1,0,Manchester City,Wolves,W,Manchester City
101,1,0,2023-02-04,Wolverhampton Wanderers,Liverpool,W,Wolves,0,0,Liverpool,Wolves,L,Liverpool


In [221]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

0    13
1    10
Name: actual_x, dtype: int64